#### 基本语法
##### init.xavier_uniform(self.weight)
- Xavier 初始化（也称为 Glorot 初始化）：用来初始化神经网络层的权重矩阵 self.weight。具体来说，xavier_uniform 是一种权重初始化方法，它生成的权重值服从一个均匀分布，该分布的范围依据输入和输出的维度自动确定，以确保权重的初始值适合神经网络的训练

##### combined = torch.cat([self_feats, neigh_feats], dim=1)
- PyTorch 中用于沿指定维度连接（拼接）两个或多个张量的函数
- arg1：一个张量列表
- arg2：指定了在哪个维度上进行拼接
- dim=1：按列拼接，即行数不变，列数增加，一行一行拼接

##### combined = F.relu(self.weight.mm(combined.t()))
- combined.t()：置换操作
- F.relu：PyTorch 中的 ReLU（Rectified Linear Unit）激活函数

##### F.normalize(combined, p=2, dim=0)
- 归一化处理
- arg1：需要进行归一化处理的张量
- arg2：指定范数类型
- arg3：维度，dim=0表示对每一列进行归一化处理，也就是在南北方向上归一
- arg4：eps，可选，在归一化过程中，如果某些向量的范数非常接近于零，可能会导致除以非常小的数值，从而引发数值不稳定性。eps 的作用就是在这种情况下增加一个很小的数，以避免除以零，默认为1e-12


In [ ]:
import torch
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F

class Encoder(nn.Module):
    """
    Encodes a node's using 'convolutional' GraphSage approach
    """
    def __init__(self, features, feature_dim, 
            embed_dim, adj_lists, aggregator,
            num_sample=10,
            base_model=None, gcn=False, cuda=False, 
            feature_transform=False): 
        super(Encoder, self).__init__()

        self.features = features
        self.feat_dim = feature_dim
        self.adj_lists = adj_lists
        self.aggregator = aggregator
        self.num_sample = num_sample
        if base_model != None:
            self.base_model = base_model

        self.gcn = gcn
        self.embed_dim = embed_dim
        self.cuda = cuda
        self.aggregator.cuda = cuda
        self.weight = nn.Parameter(
                torch.FloatTensor(embed_dim, self.feat_dim if self.gcn else 2 * self.feat_dim))
        init.xavier_uniform(self.weight)

    def forward(self, nodes):
        """
        Generates embeddings for a batch of nodes.

        nodes     -- list of nodes
        """
        neigh_feats = self.aggregator.forward(nodes, [self.adj_lists[int(node)] for node in nodes], 
                self.num_sample)
        if not self.gcn:
            if self.cuda:
                self_feats = self.features(torch.LongTensor(nodes).cuda())
            else:
                self_feats = self.features(torch.LongTensor(nodes))
            combined = torch.cat([self_feats, neigh_feats], dim=1)
        else:
            combined = neigh_feats
        combined = F.relu(self.weight.mm(combined.t()))
        # L2范数归一化
        combined = F.normalize(combined, p=2, dim=0)
        return combined
